In [2]:
from knn_chain import knn_chain

In [6]:
from scipy.cluster.hierarchy import ward
from scipy.spatial.distance import pdist

In [3]:
import numpy as np
import pstats, cProfile
import pyximport
pyximport.install()

(None, <pyximport._pyximport3.PyxImportMetaFinder at 0x7f64b07fd060>)

In [4]:
X = np.loadtxt("largeX")

## Profiling:
- knn-chain from c++, k = 6 vs k = 1  

In [4]:
cProfile.runctx("knn_chain(X, 6)", globals(), locals(), "Profile.prof")
s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Fri Jul 12 14:56:15 2024    Profile.prof

         284 function calls in 6.136 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    6.133    6.133    6.135    6.135 {built-in method knn_chain.knn_chain}
       28    0.002    0.000    0.002    0.000 ipkernel.py:770(_clean_thread_parent_frames)
        1    0.000    0.000    6.136    6.136 <string>:1(<module>)
       14    0.000    0.000    0.000    0.000 threading.py:1478(enumerate)
       98    0.000    0.000    0.000    0.000 threading.py:1145(ident)
       14    0.000    0.000    0.000    0.000 ipkernel.py:785(<setcomp>)
        1    0.000    0.000    6.136    6.136 {built-in method builtins.exec}
       56    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
       28    0.000    0.000    0.000    0.000 {method 'values' of 'dict' objects}
       14    0.000    0.000    0.000    0.000 {method '__exit__' of '_thread.RLock' objects}
       28    0.0

In [5]:
cProfile.runctx("knn_chain(X, 1)", globals(), locals(), "Profile.prof")
s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Fri Jul 12 14:56:50 2024    Profile.prof

         284 function calls in 10.109 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   10.108   10.108   10.109   10.109 {built-in method knn_chain.knn_chain}
       28    0.001    0.000    0.001    0.000 ipkernel.py:770(_clean_thread_parent_frames)
        1    0.000    0.000   10.109   10.109 <string>:1(<module>)
       14    0.000    0.000    0.000    0.000 threading.py:1478(enumerate)
        1    0.000    0.000   10.109   10.109 {built-in method builtins.exec}
       14    0.000    0.000    0.000    0.000 ipkernel.py:785(<setcomp>)
       98    0.000    0.000    0.000    0.000 threading.py:1145(ident)
       28    0.000    0.000    0.000    0.000 {method 'values' of 'dict' objects}
       56    0.000    0.000    0.000    0.000 {method 'keys' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       14    

- SciPy implementation

In [7]:
cProfile.runctx("pdist(X)", globals(), locals(), "Profile.prof")
s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()
y = pdist(X) 

Fri Jul 12 14:57:13 2024    Profile.prof

         14 function calls in 3.661 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.660    3.660    3.660    3.660 {built-in method scipy.spatial._distance_pybind.pdist_euclidean}
        1    0.001    0.001    3.661    3.661 <string>:1(<module>)
        1    0.000    0.000    3.660    3.660 distance.py:1961(pdist)
        1    0.000    0.000    3.661    3.661 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 _util.py:241(_asarray_validated)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 _base.py:1291(isspmatrix)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 {built-in method numpy.asarray

In [8]:
cProfile.runctx("ward(y)", globals(), locals(), "Profile.prof")
s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Fri Jul 12 14:57:21 2024    Profile.prof

         52 function calls in 3.862 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.770    3.770    3.778    3.778 {scipy.cluster._hierarchy.nn_chain}
        1    0.048    0.048    3.862    3.862 {built-in method builtins.exec}
        1    0.032    0.032    3.814    3.814 hierarchy.py:837(linkage)
        1    0.008    0.008    0.008    0.008 {method 'argsort' of 'numpy.ndarray' objects}
        1    0.003    0.003    0.003    0.003 {method 'reduce' of 'numpy.ufunc' objects}
        1    0.000    0.000    3.814    3.814 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 fromnumeric.py:1034(_argsort_dispatcher)
        2    0.000    0.000    0.000    0.000 distance.py:2475(is_valid_y)
        4    0.000    0.000    0.012    0.003 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.000    0.000    0.003    0.003 fromn

- prev results

In [ ]:
# PREV RESULTS FOR IMPORTED LIB
# k = 1 ==> 30 s
# k = 6 ==> 18 s

# avg = 0.0

# for i in range(10):
#     cProfile.runctx("knn_chain(X, 1)", globals(), locals(), "Profile.prof")
#     s = pstats.Stats("Profile.prof")
#     avg += s.strip_dirs().sort_stats("time").total_tt
    
# print(f"avg = {avg/10}")
# s.strip_dirs().sort_stats("time").print_stats()